<a href="https://colab.research.google.com/github/ABDA48/LangchainNotebook/blob/main/Langchain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

In [3]:
!pip install -qU langchain-groq

In [4]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"]=userdata.get('GROQ_API_KEY')


Let's first use the model directly. **ChatModels** are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the .invoke method.

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_time': 0.003333333, 'prompt_time': 0.00310181, 'queue_time': 0.01152628, 'total_time': 0.006435143}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9e8ac55-e983-4d00-81de-02232b65eba9-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

# **OutputParsers**
Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [7]:
result = model.invoke(messages)

In [10]:
parser.invoke(result)

'Ciao!'

# **Prompt Templates**
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

In [12]:
system_template = "Translate the following into {language}:"

In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [14]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [15]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

# **Chaining together components with LCEL**
We can now combine this with the model and the output parser from above using the pipe **(|)** operator:

In [16]:
chain = prompt_template | model | parser

In [17]:
chain.invoke({"language": "italian", "text": "hi"})

'Ciao!'

# **Serving with LangServe**
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

In [18]:
!pip install "langserve[all]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00


In [20]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langserve import add_routes
from fastapi import FastAPI
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)


parser = StrOutputParser()
chain = prompt_template | model | parser

# 5. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 6. Adding chain route
add_routes(
    app,
    chain,
    path="/chain",
)
"""
if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
"""

'\nif __name__ == "__main__":\n    import uvicorn\n\n    uvicorn.run(app, host="localhost", port=8000)\n'